# 머신 러닝 교과서 - 파이토치편

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/ml-with-pytorch/blob/main/ch15/ch15_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

**주의**: 책에 있는 코드를 재현하려면 이 장에서 사용한 패키지인 `torchtext` 0.10.0(https://pypi.org/project/torchtext/0.10.0/) 버전을 사용해야 합니다.

이 노트북에는 최신 버전의 `torchtext`를 사용하려면 몇 가지를 수정해야 합니다.

torchtext 0.10.0을 사용하기 위해서는 python 3.9 버전 필요!

In [1]:
%pip install torchtext==0.10.0

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------------------------------------ --- 1.3/1.4 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 7.6 MB/s  0:00:00
   ---------------------------------------- 0.0/222.0 MB ? eta -:--:--
   ---------------------------------------- 1.6/222.0 MB 9.4 MB/s eta 0:00:24
    --------------------------------------- 3.9/222.0 MB 9.8 MB/s eta 0:00:23
    --------------------------------------- 5.2/222.0 MB 8.6 MB/s eta 0:00:26
   - -------------------------------------- 6.0/222.0 MB 7.7 MB/s eta 0:00:29
   - -------------------------------------- 7.3/222.0 MB 7.2 MB/s eta 0:00:30
   - -------------------------------------- 8.7/222.0 MB 7.1 MB/s eta 0:00:31
   - -------------------------------------- 10.2/222.0 MB 7.2 MB/s eta 0:00:30
   -- ------------------------------------- 12.1/222.0 MB 7.3 MB/s eta 0:00:29
   -- ------------------------------------- 14.2/222.0 MB 7.6 MB/s eta 0:00:28
   -- 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-ignite 0.5.3 requires torch<3,>=1.10, but you have torch 1.9.0 which is incompatible.
pytorch-lightning 2.6.0 requires torch>=2.1.0, but you have torch 1.9.0 which is incompatible.
torchmetrics 1.8.2 requires torch>=2.0.0, but you have torch 1.9.0 which is incompatible.
torchvision 0.23.0 requires torch==2.8.0, but you have torch 1.9.0 which is incompatible.


최신 버전의 `torchtext`를 사용하려면 `portalocker`가 필요합니다:

In [2]:
%pip install portalocker==2.1.0

Note: you may need to restart the kernel to use updated packages.


권장 패키지 버전을 확인하세요:

In [3]:
from python_environment_check import check_packages


d = {
    'torch': '1.8.0',
    'torchtext': '0.10.0'
}
check_packages(d)

[OK] Your Python version is 3.9.25 (main, Nov  3 2025, 22:44:01) [MSC v.1929 64 bit (AMD64)]



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\user\miniconda3\envs\torchlightning\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\user\miniconda3\envs\torchlightning\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\user\miniconda3\envs\torchlightning\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\user\miniconda3\envs\torchlightning\lib\site-packages\traitlets\config\application.py", line 1075, 

[OK] torch 1.9.0+cpu
[OK] torchtext 0.10.0


# 15장 - 순환 신경망으로 순차 데이터 모델링 (파트 2/3)

**목차**

- 파이토치로 시퀀스 모델링을 위한 RNN 구현
  - 첫 번째 프로젝트: IMDb 영화 리뷰의 감성 분석
    - 영화 리뷰 데이터 준비
    - 문장 인코딩을 위한 임베딩 층
    - RNN 모델 만들기
    - 감성 분석 작업을 위한 RNN 모델 만들기
      - 양방향 RNN

In [4]:
from IPython.display import Image

# 15.3 파이토치로 시퀀스 모델링을 위한 RNN 구현

## 15.3.1 첫 번째 프로젝트: IMDb 영화 리뷰의 감성 분석

- 감성분석 : 텍스트 > 다대일 구조 RNN

### 영화 리뷰 데이터 준비

In [5]:
import torch
import torch.nn as nn

In [6]:
import torchtext
print(torchtext.__version__)

0.10.0


In [7]:
from torchtext.datasets import IMDB
from torch.utils.data.dataset import random_split

# 단계 1: 데이터셋을 로드합니다

train_dataset = IMDB(split='train') # iterator / iterable dataset
test_dataset = IMDB(split='test')

# test_dataset = list(test_dataset)   #datapipe to list
## iterable 객체는 한번만 순회 > 여러번 순회하려면 list() 사용해야 

torch.manual_seed(1)
# 훈련 데이터를 훈련, 검증 데이터로 분할
train_dataset, valid_dataset = random_split(
    list(train_dataset), [20000, 5000]) # 훈련 20000개, 검증 5000개
# random_split()은 train_dataset을 여러번 접근하기 때문이다 

c:\workspace_deep_learning\.data\IMDB\aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [05:32<00:00, 253kB/s] 


In [8]:
head5 = [train_dataset[i] for i in range(5)]
head5
# label, text

[('pos',
  'An extra is called upon to play a general in a movie about the Russian Revolution. However, he is not any ordinary extra. He is Serguis Alexander, former commanding general of the Russia armies who is now being forced to relive the same scene, which he suffered professional and personal tragedy in, to satisfy the director who was once a revolutionist in Russia and was humiliated by Alexander. It can now be the time for this broken man to finally "win" his penultimate battle. This is one powerful movie with meticulous direction by Von Sternberg, providing the greatest irony in Alexander\'s character in every way he can. Jannings deserved his Oscar for the role with a very moving performance playing the general at his peak and at his deepest valley. Powell lends a sinister support as the revenge minded director and Brent is perfect in her role with her face and movements showing so much expression as Jannings\' love. All around brilliance. Rating, 10.'),
 ('pos',
  "almost ev

In [9]:
## 단계 2: 고유 토큰 (단어) 찾기
## IMDB 학습 데이터로부터 “단어 빈도 사전(vocabulary statistics)”을 만드는 전형적인 전처리 파이프라인
import re
from collections import Counter, OrderedDict

token_counts = Counter() # 단어별 횟수 계산, Counter는 dict의 하위 클래스

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text) #text에서 <로 시작해서 >로 끝나는 모든 태그를 제거
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower()) # 감성 이모티콘: 눈, 코, 입
    text = re.sub('[\W]+', ' ', text.lower()) +' '.join(emoticons).replace('-', '') # 특수 문자를 공백으로
    # \W: 알파벳/숫자/언더스코어가 아닌 문자
    tokenized = text.split()
    return tokenized


for label, line in train_dataset:
    tokens = tokenizer(line)
    token_counts.update(tokens) # 토큰 빈도 수


print('어휘 사전 크기:', len(token_counts))

어휘 사전 크기: 69023


token_counts = {
    "the": 231358,
    "movie": 58923,
    "good": 41235,
    "bad": 39821,
    ...
}


token_counts.items() > (token, freq) 튜플들

- dict 관련 핵심 메소드 정리:

 for k in d.keys(), for v in d.values(), for k, v in d.items()

In [10]:
for token, freq in token_counts.items():
    print(token, freq)


an 17204
extra 244
is 85847
called 1140
upon 675
to 107513
play 1752
a 130057
general 619
in 74646
movie 35149
about 13734
the 267877
russian 250
revolution 160
however 2892
he 24075
not 24329
any 6089
ordinary 222
serguis 2
alexander 100
former 416
commanding 37
of 116119
russia 66
armies 17
who 17112
now 3704
being 5224
forced 505
relive 19
same 3244
scene 4223
which 9563
suffered 116
professional 270
and 130797
personal 510
tragedy 291
satisfy 73
director 3578
was 38365
once 1886
revolutionist 1
humiliated 23
by 17895
it 76964
can 11688
be 21260
time 10042
for 35262
this 60714
broken 220
man 4785
finally 1216
win 373
his 23434
penultimate 13
battle 505
one 21383
powerful 503
with 35163
meticulous 11
direction 1101
von 159
sternberg 12
providing 95
greatest 578
irony 116
s 50537
character 5485
every 3189
way 6428
jannings 23
deserved 235
oscar 690
role 2506
very 11229
moving 675
performance 2298
playing 1290
at 18751
peak 62
deepest 39
valley 64
powell 160
lends 47
sinister 133
suppo

movie 12000
good 8000
bad 7000


In [11]:
## 단계 3: 고유 토큰을 정수로 인코딩하기
from torchtext.vocab import vocab
# items()는 딕셔너리(또는 Counter)에 들어 있는 (key, value) 쌍을 한 번에 꺼내주는 함수
sorted_by_freq_tuples = sorted(token_counts.items(), key=lambda x: x[1], reverse=True) # 정렬을 빈도수로
ordered_dict = OrderedDict(sorted_by_freq_tuples) # 정렬된 튜플 리스트를 순서가 보존된 딕셔너리로 변환

vocab = vocab(ordered_dict) # 인덱스 매핑 생성

vocab.insert_token("<pad>", 0) # 패딩 토큰
vocab.insert_token("<unk>", 1)
vocab.set_default_index(1)

print([vocab[token] for token in ['this', 'is', 'an', 'example']]) # vocab["movie"]는 인덱스를 리턴
# 텍스트를 인덱스로 바꾼후 LSTM 입력 전체 파이프라인

[11, 7, 35, 457]


In [12]:
if not torch.cuda.is_available():
    print("경고: 이 코드는 CPU에서 매우 느릴 수 있습니다.")

경고: 이 코드는 CPU에서 매우 느릴 수 있습니다.


In [13]:
## 단계 3-A: 변환 함수 정의
import torchtext

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#x는 텍스트
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)] # 토큰에 대한 정수 인덱스
# 텍스트 → 토큰 → 인덱스 변환 규칙을 하나의 ‘함수 객체’로 정의
# def text_pipeline(x):
#    return [vocab[token] for token in tokenizer(x)]

# 예: ["I","love","it"] → [12, 534, 87] 같은 정수 리스트
from torchtext import __version__ as torchtext_version
from pkg_resources import parse_version

if parse_version(torchtext.__version__) > parse_version("0.10"):
    label_pipeline = lambda x: 1. if x == 2 else 0.         # 1 ~ 부정 리뷰, 2 ~ 긍정 리뷰
else:
    label_pipeline = lambda x: 1. if x == 'pos' else 0.
# 긍정: 1.0, 부정: 0.0

## 단계 3-B: 인코딩과 변환 함수 감싸기
# 가변 길이 텍스트 배치를 모델이 바로 받을 수 있는 텐서 형태로 묶어 주는 함수
def collate_batch(batch): #batch는 미니배치 텍스트
    label_list, text_list, lengths = [], [], []
    for _label, _text in batch: # batch = [(label1, text1), (), ...]
        label_list.append(label_pipeline(_label))
        # label_pipeline: 원본 라벨 → 0.0 / 1.0
        processed_text = torch.tensor(text_pipeline(_text),
                                      dtype=torch.int64)
        # text_pipeline(_text) 문자열 → 토큰화 → vocab 인덱스 리스트
        ## "i love this movie"
        ## → ["i","love","this","movie"]
        ## → [12, 534, 78, 921]

        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
    label_list = torch.tensor(label_list)
    lengths = torch.tensor(lengths)
    padded_text_list = nn.utils.rnn.pad_sequence(
        text_list, batch_first=True)
    # 길이가 다른 시퀀스들을 같은 길이로 맞춤
    # 짧은 문장은 PAD 토큰(0) 으로 뒤를 채움
    return padded_text_list.to(device), label_list.to(device), lengths.to(device)

C:\Users\user\AppData\Local\Temp\ipykernel_26580\762657912.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [ ]:
## 작은 배치를 만듭니다.

from torch.utils.data import DataLoader
dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False, collate_fn=collate_batch)
# collate_batch()는 DataLoader가 “배치 하나를 만들 때마다 자동으로 호출”하는 함수이며,
## 여러 개의 개별 샘플을 → 하나의 배치 텐서로 묶는 역할
text_batch, label_batch, length_batch = next(iter(dataloader)) # collate_fn(batch) 자동 호출
# next(iter(dataloader))는 DataLoader에서 “첫 번째 배치”를 하나 생성하고,
## collate_fn의 반환값을 그대로 받아오는 동작
print(text_batch)
print(label_batch)
print(length_batch)
print(text_batch.shape)

tensor([[   35,  1739,     7,   449,   721,     6,   301,     4,   787,     9,
             4,    18,    44,     2,  1705,  2460,   186,    25,     7,    24,
           100,  1874,  1739,    25,     7, 34415,  3568,  1103,  7517,   787,
             5,     2,  4991, 12401,    36,     7,   148,   111,   939,     6,
         11598,     2,   172,   135,    62,    25,  3199,  1602,     3,   928,
          1500,     9,     6,  4601,     2,   155,    36,    14,   274,     4,
         42945,     9,  4991,     3,    14, 10296,    34,  3568,     8,    51,
           148,    30,     2,    58,    16,    11,  1893,   125,     6,   420,
          1214,    27, 14542,   940,    11,     7,    29,   951,    18,    17,
         15994,   459,    34,  2480, 15211,  3713,     2,   840,  3200,     9,
          3568,    13,   107,     9,   175,    94,    25,    51, 10297,  1796,
            27,   712,    16,     2,   220,    17,     4,    54,   722,   238,
           395,     2,   787,    32,    27,  5236,  

In [15]:
## 단계 4: 데이터셋 배치 만들기

batch_size = 32

train_dl = DataLoader(train_dataset, batch_size=batch_size,
                      shuffle=True, collate_fn=collate_batch)
valid_dl = DataLoader(valid_dataset, batch_size=batch_size,
                      shuffle=False, collate_fn=collate_batch)
test_dl = DataLoader(test_dataset, batch_size=batch_size,
                     shuffle=False, collate_fn=collate_batch)

### 문장 인코딩을 위한 임베딩 층

 * `input_dim`: 단어 수, i.e. 정수 인덱스 최댓값 + 1.

 * `output_dim`: 

 * `input_length`: (패딩된) 시퀀스 길이 > 문장의 실제 길이가 아님 > 입력 크기 10으로 하므로 0으로 패딩

    * 예를 들어, `'This is an example' -> [0, 0, 0, 0, 0, 0, 3, 1, 8, 9]`   

    => input_lenght는 10

 * 층을 호출할 때 정수 값을 입력으로 받고 임베딩 층이 정수를 `[output_dim]` 크기의 실수 벡터로 바꿉니다.

   * 입력 크기가 `[BATCH_SIZE]`이면, 출력 크기는 `[BATCH_SIZE, output_dim]`가 됩니다
   
   * 입력 크기가 `[BATCH_SIZE, 10]`이면, 출력 크기는 `[BATCH_SIZE, 10, output_dim]`가 됩니다.

입력 시퀀스 : 단어 인덱스를 입력 특성

1) one-hot encoding

 - 전체 데이터세트의 단어 수에 해당하는 크기의 벡터를 만듬

  > 전체 단어 수가 많으면 벡터의 차원/특성이 단어수 > 차원의 저주

2) 임베딩

 - 각 단어를 실수 값을 갖는 고정된 길이의 벡터로 변환

 - 임베딩은 ‘미리 정해주는 값’이 아니라, 신경망이 손실(loss)을 줄이기 위해 가중치를 업데이트하는 과정에서 자연스럽게 결정되는 학습 파라미터

신경망만으로 임베딩을 부여한다

- 임베딩은 입력이지만, 동시에 학습되는 가중치(parameter)

- 임베딩 벡터들이 LSTM / CNN / Linear 등을 통과

  > 예측값  𝑦^  생성 > 예측 값은 감성분류 값 등

  > 손실계산: loss = L(y^,y)

  > 역전파(임베딩 학습): ​∂L/∂ew !=0 : ew는 단어 w에 대한 임베딩

  > SGD: ew <- ew - m ​∂L/∂ew

  > 신경망 학습으로 임베딩 값이 부여된다

임베딩을 신경망으로 학습한다는 말은 “단어를 숫자로 미리 정해두는 것이 아니라,

예측을 잘 하도록 숫자(벡터)를 신경망 학습으로 계속 고쳐 가며 배우게 한다”

단어 빈도(BOW(Bag of Words), TF-IDF)

movie → [0,0,1,0,0,...]
good  → [0,1,0,0,0,...]

 - 의미를 모름

 - good ↔ great ↔ excellent 관계 전혀 모름

 - 순서·문맥 무시

신경망 학습 - 임베딩

1. 단어 → 임베딩 벡터

 "movie" → e_movie = [0.3, -1.2, 0.7, ...]

 - 처음엔 랜덤 값
 

2. 임베딩이 신경망을 통과
 
 - 임베딩 → LSTM / CNN / Linear → 예측값 ŷ

 - y^: 감성 분류, 주제 분류, 다음 단어 예측에 사용됨

3. 손실 계산

 - loss = L(ŷ, y)

 - 역전파가 “임베딩까지” 내려온다: dL/dew != 0

4. SGD로 임베딩 자체를 수정

 - ew = ew - mu . dL/dew > 단어의 벡터를 업데이트

감성 분류: “great”, “excellent”가 자주 **같은 라벨(긍정)**에 기여

- 모델은 loss를 줄이기 위해

- 이 단어들의 임베딩을 비슷한 방향으로 이동

"good" ≈ [ 1.3,  0.9]
"great"≈ [ 1.2,  1.0]

"bad"  ≈ [-1.1, -0.8]
"awful"≈ [-1.2, -0.9] # awful 뜻: 아주 나쁜, 끔찍한 > 감정의 강도: bad > terrible > awful

- 차원 하나하나는 의미 없음

 > 벡터의 방향·거리가 의미

 > 임베딩은 신경망 학습 과정에서 손실 함수를 최소화하도록

  + 다른 가중치들과 동일하게 역전파로 업데이트되며,

  + 그 결과 과제 수행에 유용한 의미적 특성이 벡터 공간에 반영된다.

In [14]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch15/figures/15_10.png', width=600)

임베딩(nn.Embedding())이 하는 일:

 - 정수 → 벡터 변환

 - 벡터는 학습 과정에서 자동으로 조정

 - 의미가 비슷한 단어 → 벡터도 비슷해짐

In [16]:
embedding = nn.Embedding(num_embeddings=10, # 임베딩 행의 수 > 단어 수
                         embedding_dim=3,
                         padding_idx=0)

# 네 개의 인덱스를 가진 두 개의 샘플로 구성된 배치
text_encoded_input = torch.LongTensor([[1,2,4,5],[4,3,2,0]]) # 단어들의 정수 인코딩
print(embedding(text_encoded_input))

tensor([[[ 0.7039, -0.8321, -0.4651],
         [-0.3203,  2.2408,  0.5566],
         [-0.4643,  0.3046,  0.7046],
         [-0.7106, -0.2959,  0.8356]],

        [[-0.4643,  0.3046,  0.7046],
         [ 0.0946, -0.3531,  0.9124],
         [-0.3203,  2.2408,  0.5566],
         [ 0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward>)


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim

# ----------------------------
# 1. 단어 사전
# ----------------------------
word2idx = {
    "<PAD>": 0,
    "good": 1,
    "bad": 2,
    "movie": 3
}

# ----------------------------
# 2. 입력 데이터 (정수 시퀀스)
# ----------------------------
# 문장: "good movie", "bad movie"
X = torch.tensor([
    [1, 3],  # good movie
    [2, 3]   # bad movie
])

y = torch.tensor([1.0, 0.0])  # 긍정 / 부정

# ----------------------------
# 3. 모델 정의
# ----------------------------
embedding = nn.Embedding(num_embeddings=4, embedding_dim=2, padding_idx=0)

model = nn.Sequential(
    embedding,
    nn.Flatten(),     # (B, 2, 2) → (B, 4)
    nn.Linear(4, 1),
    nn.Sigmoid()
)

# ----------------------------
# 4. 학습 준비
# ----------------------------
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.5)

# ----------------------------
# 5. 학습 전 임베딩 확인
# ----------------------------
print("=== 학습 전 임베딩 ===")
print("good :", embedding.weight[1].detach())
print("bad  :", embedding.weight[2].detach())
print("movie :", embedding.weight[3].detach())
print()

# ----------------------------
# 6. 학습
# ----------------------------
for epoch in range(20):
    optimizer.zero_grad()
    output = model(X).squeeze()
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()

# ----------------------------
# 7. 학습 후 임베딩 확인
# ----------------------------
print("=== 학습 후 임베딩 ===")
print("good :", embedding.weight[1].detach())
print("bad  :", embedding.weight[2].detach())
print("movie :", embedding.weight[3].detach())


=== 학습 전 임베딩 ===
good : tensor([-0.1672, -0.9789])
bad  : tensor([-0.8047,  0.7838])
movie : tensor([ 0.7138, -1.1160])

=== 학습 후 임베딩 ===
good : tensor([ 0.6249, -1.3717])
bad  : tensor([-1.5405,  1.1258])
movie : tensor([ 0.7197, -1.1079])


### RNN 모델 만들기

* **RNN layers:**
  * `nn.RNN(input_size, hidden_size, num_layers=1)`
  * `nn.LSTM(..)`
  * `nn.GRU(..)`
  * `nn.RNN(input_size, hidden_size, num_layers=1, bidirectional=True)`

LSTM(Long Short-Term Memory) 개발자: 1997년에 제안.

 - Sepp Hochreiter, Jürgen Schmidhuber

 - 독일/스위스 계열 연구 그룹

GRU(Gated Recurrent Unit):

- 뉴욕대학교의 조경현(Kyunghyun Cho) 교수 등 연구진이 제안한 모델, 2014(LSTM 개발자와 관련없다)

In [22]:
## 간단한 RNN 층을 사용한 RNN 모델 구축 예제

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.rnn = nn.RNN(input_size,
                          hidden_size,
                          num_layers=2,
                          batch_first=True)
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1) # 출력층

    def forward(self, x):
        _, hidden = self.rnn(x)
        out = hidden[-1, :, :] # hidden[-1] → 2층 중 마지막 층의 은닉 상태
        # hidden.shape = (num_layers, batch_size, hidden_size)
        # hidden은 모든 층(layer) 에 대해
        # 마지막 시점(t = T) 의 은닉 상태를 담고 있다.
        out = self.fc(out)
        return out

model = RNN(64, 32)

print(model)

model(torch.randn(5, 3, 64))

RNN(
  (rnn): RNN(64, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)


tensor([[-0.0702],
        [-0.1757],
        [ 0.1173],
        [ 0.1923],
        [-0.2402]], grad_fn=<AddmmBackward>)

왜 many-to-one 문제에서는 hidden[-1]인가?

- 입력: 시퀀스 전체 (문장, 시계열)

- 출력: 하나의 값

 > 감성 분류

 > 문장 분류

 > 시계열 예측 1개 값

RNN 내부 계산:

 ht = f(ht-1,xt)

 마지막 시점: hT = f(hT-1, xT) > hT에는 x1 ~ xT의 정보가 누적되어 있다 

LSTM에는 역할이 완전히 다른 두 종류의 값이 있다.

- 무엇을 기억/표현: ct, ht > cell 상태로 표현

  ct = ft.ct-1 + it.ct~

  ht = ot.tanh(ct)

- 얼마나 유지/추가/노출할 것인가를 조절: ft, it, ot

### 감성 분석 작업을 위한 RNN 모델 만들기

- 가변 길이 텍스트(패딩 포함)를 입력으로 받아 LSTM으로 문장(리뷰) 전체를 요약

- 이진 분류(긍/부정) 확률을 출력하는 many-to-one 모델

In [31]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size,
                                      embed_dim,
                                      padding_idx=0)
        # 입력은 text가 정수 인덱스 시퀀스(예: [12, 53, 7, ...])
        # nn.Embedding은 이를 실수 벡터 시퀀스로 변환
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size,
                           batch_first=True)
        # LSTM이 만든 “문장 요약 벡터”를 받는다 
        self.fc1 = nn.Linear(rnn_hidden_size, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text) #text: 패딩된 문장 인덱스
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu(), enforce_sorted=False, batch_first=True)
        # 각 문장마다 실제 길이까지만 RNN이 처리
        # PAD 부분은 아예 계산에서 제외됨(효율 + 정확성)

        out, (hidden, cell) = self.rnn(out)
        # out은 각 시점들의 ht를 모은 것 
        # hidden[-1] : 마지막 층의 마지막 시점 요약 → 필요
        out = hidden[-1, :, :]
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

vocab_size = len(vocab)
embed_dim = 20
rnn_hidden_size = 64
fc_hidden_size = 64

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size)
model = model.to(device)

In [34]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [35]:
def train(dataloader):
    # 학습용 DataLoader를 받아서 1 epoch 학습을 수행하고
    # 정확도(acc), 평균 손실(loss) 을 반환하는 함수
    model.train()
    total_acc, total_loss = 0, 0
    for text_batch, label_batch, lengths in dataloader:
        optimizer.zero_grad()
        pred = model(text_batch, lengths)[:, 0]
        loss = loss_fn(pred, label_batch)
        loss.backward()
        optimizer.step()
        total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
        total_loss += loss.item()*label_batch.size(0)
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset)

def evaluate(dataloader):
    model.eval()
    total_acc, total_loss = 0, 0
    with torch.no_grad():
        for text_batch, label_batch, lengths in dataloader:
            pred = model(text_batch, lengths)[:, 0]
            loss = loss_fn(pred, label_batch)
            total_acc += ((pred>=0.5).float() == label_batch).float().sum().item()
            total_loss += loss.item()*label_batch.size(0)
    return total_acc/len(list(dataloader.dataset)), total_loss/len(list(dataloader.dataset))

In [28]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [29]:
import numpy as np

In [ ]:


num_epochs = 10

torch.manual_seed(1)

for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dl)
    acc_valid, loss_valid = evaluate(valid_dl)
    print(f'에포크 {epoch} 정확도: {acc_train:.4f} 검증 정확도: {acc_valid:.4f}')

In [25]:
acc_test, _ = evaluate(test_dl)
print(f'테스트 정확도: {acc_test:.4f}')

테스트 정확도: 0.8494


#### 양방향 RNN

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size,
                                      embed_dim,
                                      padding_idx=0)
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size,
                           batch_first=True, bidirectional=True)
        # bidirectional=True는 양방향 RNN(Bidirectional LSTM)을 구동
        # 내부적으로 RNN이 2개 생김
        # Forward LSTM   : t=1 → T
        # Backward LSTM  : t=T → 1
        # output의 shape: output.shape = (batch_size, seq_len, hidden_size * 2)

        self.fc1 = nn.Linear(rnn_hidden_size*2, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu(), enforce_sorted=False, batch_first=True)
        _, (hidden, cell) = self.rnn(out)
        out = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size)
model = model.to(device)

올바른 대표 벡터 추출 방법:

# forward + backward를 concat

h_forward  = hidden[-2]

h_backward = hidden[-1]

out = torch.cat([h_forward, h_backward], dim=1)



In [27]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

num_epochs = 10

torch.manual_seed(1)

for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dl)
    acc_valid, loss_valid = evaluate(valid_dl)
    print(f'에포크 {epoch} 정확도: {acc_train:.4f} 검증 정확도: {acc_valid:.4f}')

에포크 0 정확도: 0.6254 검증 정확도: 0.7030
에포크 1 정확도: 0.7802 검증 정확도: 0.7444
에포크 2 정확도: 0.8495 검증 정확도: 0.8462
에포크 3 정확도: 0.8912 검증 정확도: 0.8384
에포크 4 정확도: 0.9315 검증 정확도: 0.8546
에포크 5 정확도: 0.9522 검증 정확도: 0.8546
에포크 6 정확도: 0.9700 검증 정확도: 0.8700
에포크 7 정확도: 0.9792 검증 정확도: 0.8448
에포크 8 정확도: 0.9857 검증 정확도: 0.8668
에포크 9 정확도: 0.9943 검증 정확도: 0.8664


In [28]:
test_dataset = IMDB(split='test')
test_dl = DataLoader(test_dataset, batch_size=batch_size,
                     shuffle=False, collate_fn=collate_batch)

In [29]:
acc_test, _ = evaluate(test_dl)
print(f'테스트 정확도: {acc_test:.4f}')

테스트 정확도: 0.8480
